# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
def get_gini(df, label):
    a=df[label].value_counts()/len(df[label])
    # P_2는 확률의 제곱을 뜻함. 뒤에서 루프를 돌려 더할 것이므로 0이라는 초기값 설정해두기
    P_2=0 
    
    # 레이블 값이 두개가 아닌 그 이상일 수 있기 때문에 P_2에 확률의 제곱의 합을 대입해두기
    for i in range(len(a)):
        P_2+=a[i]**2
    
    # 지니계수 계산
    gini= 1-(P_2)
    return gini

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [5]:
def get_binary_split(df, attribute):
    from itertools import combinations
    unique_val=df[attribute].unique()
    list_t=[] # 리스트 안 원소들이 튜플로 이루어진 경우
    result=[] # 리스트 안 원소들도 리스트로 이루어진 경우 (정답은 이것이므로 list_t를 result로 바꿔주는 작업 필요)
    for i in range(1,len(unique_val)):
        list_t.extend(list(combinations(unique_val,i)))
    for i in range(len(list_t)):
        result.append(list(list_t[i]))
    return result

In [6]:
get_binary_split(pd_data,"income")

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_attribute_gini_index(df, attribute, label):
    # get_binary_split()을 통해 attribute 컬럼의 조합을 binary변수로 받기
    binary=get_binary_split(df,attribute)
    # 결과가 출력될 빈 딕셔너리
    result={}
    
    for col in binary:
        # 부분집합의 원소가 하나인 경우
        if len(col)==1:
            df_split_1=df[df[attribute]==col[0]]        # 해당 원소를 가진 행 데이터를 df_split_1이라는 데이터프레임에 저장
            df_split_2=df[~(df[attribute]==col[0])]     # df에서 df_split_1을 뺀 나머지 데이터프레임을 df_split_2에 저장
            
        # 부분집합의 원소가 두개인 경우
        else:
            df_split_1=df[(df[attribute]==col[0])|(df[attribute]==col[1])]
            df_split_2=df[~((df[attribute]==col[0])|(df[attribute]==col[1]))]
        
        # 각 데이터프레임 label의 지니계수를 계산
        gini_1=get_gini(df_split_1,label)
        gini_2=get_gini(df_split_2,label)
        
        # split될 각 노드의 지니계수를 계산
        gini_index=gini_1*(len(df_split_1)/len(df)) + gini_2*(len(df_split_2)/len(df))
        
        key="_".join(col)
        result[key]=gini_index
    return result

In [8]:
get_attribute_gini_index(pd_data, "income", "class_buys_computer")

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 보여주세요.

In [9]:
min(get_attribute_gini_index(pd_data, "income", "class_buys_computer").items())

('high', 0.4428571428571429)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [10]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for col in features:
    print(f"Minimum Gini index of {col} :",round(min(get_attribute_gini_index(pd_data, col, "class_buys_computer").values()),4))

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


In [11]:
# age 변수를 어떤 기준으로 나눴을 때, 가장 지니계수가 가장 작아지는지 확인
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

## 위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [12]:
df_1=pd_data[pd_data['age']=='middle_aged']
df_2=pd_data[~(pd_data['age']=='middle_aged')]

In [13]:
# 각 변수를 대상으로 반복문 수행
for col in features[1:]:
    print(f"Age : Minimum Gini index of {col} :",round(min(get_attribute_gini_index(df_1, col, "class_buys_computer").values()),4))
print("#############################################################")
for col in features[1:]:
    print(f"Age : Minimum Gini index of {col} :",round(min(get_attribute_gini_index(df_2, col, "class_buys_computer").values()),4))

Age : Minimum Gini index of income : 0.0
Age : Minimum Gini index of student : 0.0
Age : Minimum Gini index of credit_rating : 0.0
#############################################################
Age : Minimum Gini index of income : 0.375
Age : Minimum Gini index of student : 0.32
Age : Minimum Gini index of credit_rating : 0.4167


## Entropy 를 구하는 함수 만들기

In [31]:
def getEntropy(df, feature) :
    import numpy as np
    a=df[label].value_counts()/len(df[label])
    # P_2는 확률의 제곱을 뜻함. 뒤에서 루프를 돌려 더할 것이므로 0이라는 초기값 설정해두기
    entropy=0 
    
    # 레이블 값이 두개가 아닌 그 이상일 수 있기 때문에 val에 확률*log2확률의 합을 저장하기
    for i in range(len(a)):
        entropy-=a[i]*np.log2(a[i])
    return(entropy)

In [32]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [71]:
def getGainA(df, feature) :
    info_D = getEntropy(df, feature) # 목표변수에 대한 Entropy 를 구하기
    columns = list(df.loc[:, df.columns != feature]) # 목표변수(feature)를 제외한 나머지 변수들의 변수명을 리스트 형태로 저장
    # 결과가 출력될 result라는 변수명의 딕셔너리 생성
    result={}
    
    # val_entropy는 E('특정 컬럼'='해당 컬럼의 원소1'), E('특정 컬럼'='해당 컬럼의 원소2') 등의 엔트로피값을 리스트 형태로 모아둘 변수
    # ex) E(income=high), E(income=medium), E(income=low) 의 값을 리스트로 저장
    val_entropy=[]
    
    # weight는 컬럼별로 원소들이 가지는 가중치를 리스트 형태로 모아둘 변수
    # ex) income 변수의 경우, weight=[4/14, 6/14, 4/14]
    weight=[]
    
    for col in columns:
        for val in df[col].unique():
            df_split=df[df[col]==val]
            val_entropy.append(getEntropy(df_split,col))
            weight.append(len(df[df[col]==val])/len(df))
        
        # info_col은 해당 컬럼 전체의 엔트로피 (가중치를 고려한 평균, ex) I(income))
        info_col=np.dot(np.array(weight),np.array(val_entropy).T)
        
        # 정보이득지수 info_gain
        Info_gain=info_D-info_col
        result[col]=Info_gain
        
        # 다음 루프를 위해 두 리스트 원소는 모두 지워주기
        val_entropy.clear()
        weight.clear()
    return(result)

In [72]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}